In [1]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [2]:
#load data
df = pd.read_excel(open('MP_scenarios.xlsx', 'rb'), sheet_name='Sheet1')
df.head()

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value
0,1,55.48,120,160,194,223,15,1421,13.870
1,2,53.68,115,149,171,197,15,2396,13.420
2,3,61.56,140,191,207,459,15,2544,15.390
3,4,65.72,115,151,242,278,15,1316,16.430
4,5,64.98,120,173,221,335,15,1377,16.245


In [3]:
# define variables in first row of markdown pricing problem
num_weeks_each_disc = 4
demand = [120,160,194,223]
discount = [0, 0.15, 0.3, 0.5]
weeks = 15
price = 55.48
starting_inventory = 1421
discount_price = [price*(1-discount[i]) for i in range(num_weeks_each_disc)]
discount_price

[55.48, 47.157999999999994, 38.836, 27.74]

In [4]:
# solve one instance of markdown pricing problem

#declare model
model = ConcreteModel()

#declare DVs
model.x = Var(range(num_weeks_each_disc), domain = NonNegativeReals)

#specify the objective function -- could not figure out how to incorporate salvage value
model.Objective = Objective(expr = sum(discount_price[i]*(demand[i]*model.x[i]) for i in range(num_weeks_each_disc)), sense = maximize)

#specify the constraints
model.Constraint_weeks = Constraint(expr = sum(model.x[i] for i in range(num_weeks_each_disc)) <= weeks)
model.Constraint_inventory = Constraint(expr = sum(demand[i]*model.x[i] for i in range(num_weeks_each_disc)) <= starting_inventory)

opt = SolverFactory('glpk')
opt.solve(model)

print('Max Total Revenue =', model.Objective())
for i in range(num_weeks_each_disc):
    print(model.x[i],":",value(model.x[i]))

Max Total Revenue = 78837.08000000022
x[0] : 11.8416666666667
x[1] : 0.0
x[2] : 0.0
x[3] : 0.0


In [5]:
# put model into a function

def solve(price, demand, discount, weeks, starting_inventory):
    discount_price = [price*(1-discount[i]) for i in range(num_weeks_each_disc)]
    
    #declare model
    model = ConcreteModel()

    #declare DVs
    model.x = Var(range(num_weeks_each_disc), domain = NonNegativeReals)

    #specify the objective function
    model.Objective = Objective(expr = sum(discount_price[i]*(demand[i]*model.x[i]) for i in range(num_weeks_each_disc)), sense = maximize)

    #specify the constraints
    model.Constraint_weeks = Constraint(expr = sum(model.x[i] for i in range(num_weeks_each_disc)) <= weeks)
    model.Constraint_inventory = Constraint(expr = sum(demand[i]*model.x[i] for i in range(num_weeks_each_disc)) <= starting_inventory)

    opt = SolverFactory('glpk')
    opt.solve(model)   
    solution = []
    solution.append('Max Total Revenue = ' + str(model.Objective()))
    for i in range(num_weeks_each_disc):
        solution.append(model.x[i].value)
    return solution


In [6]:
# test function

num_weeks_each_disc = 4
demand = [120,160,194,223]
discount = [0, 0.15, 0.3, 0.5]
weeks = 15
price = 55.48
starting_inventory = 1421
solve(price, demand, discount, weeks, starting_inventory)

['Max Total Revenue = 78837.08000000022', 11.8416666666667, 0.0, 0.0, 0.0]

In [7]:
# use markdown pricing function with data read from the file

k = 1
rowdata = df.iloc[k].values.tolist()
price = rowdata[1]
demand = rowdata[2:6]
discount = [0, 0.15, 0.3, 0.5]
weeks = rowdata[6]
starting_inventory = rowdata[7]
solve(price, demand, discount, weeks, starting_inventory)

['Max Total Revenue = 101978.58', 0.0, 15.0, 0.0, 0.0]

In [8]:
# solve each of the markdown pricing file and append the solution in a new column

outputs = []
for k in range(len(df)):
    rowdata = df.iloc[k].values.tolist()
    price = rowdata[1]
    demand = rowdata[2:6]
    discount = [0, 0.15, 0.3, 0.5]
    weeks = rowdata[6]
    starting_inventory = rowdata[7]
    outputs.append(solve(price, demand, discount, weeks, starting_inventory))

In [10]:
# add solutions into dataset

df['solution'] = outputs
df

,No,Price,Demand (0% discount),Demand (15% discount),Demand (30% discount),Demand (50% discount),Weeks Left,Inventory Remaining,Salvage Value,solution
0,1,55.48,120,160,194,223,15,1421,13.870,"[Max Total Revenue = 78837.08000000022, 11.841..."
1,2,53.68,115,149,171,197,15,2396,13.420,"[Max Total Revenue = 101978.58, 0.0, 15.0, 0.0..."
2,3,61.56,140,191,207,459,15,2544,15.390,"[Max Total Revenue = 141254.12752941175, 6.294..."
3,4,65.72,115,151,242,278,15,1316,16.430,"[Max Total Revenue = 86487.52000000027, 11.443..."
4,5,64.98,120,173,221,335,15,1377,16.245,"[Max Total Revenue = 89477.46, 11.475, 0.0, 0...."
...,...,...,...,...,...,...,...,...,...,...
195,196,58.18,105,152,160,454,15,2320,14.545,"[Max Total Revenue = 113506.48291390749, 0.0, ..."
196,197,58.58,125,169,177,298,15,1465,14.645,"[Max Total Revenue = 85819.70000000001, 11.72,..."
197,198,50.18,145,218,263,362,15,1433,12.545,"[Max Total Revenue = 71907.94000000005, 9.8827..."
198,199,59.58,140,185,224,292,15,1355,14.895,"[Max Total Revenue = 80730.90000000001, 9.6785..."


In [22]:
#export dataset with solutions to csv

df.to_csv('P2_Keith_Hines.csv', index = 'No')